In [ ]:
# Install fork of spint with modifications to accept multiple cost variables
# !pip install git+https://github.com/chesterharvey/spint

In [62]:
from modeling import *
from feature_dev import *
from utils import configs

%load_ext autoreload
%autoreload 2

df = develop_features()
# df.columns.tolist()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
8190 initial records
8190 complete records from load_travel_time_and_fares
8190 complete records from load_track_miles
8190 complete records from load_am_ridership
8190 complete records from load_pm_ridership
8190 complete records from load_off_peak_ridership
8190 complete records from calculate_total_ridership_and_parking_users
8190 complete records from load_bus_travel_time
8190 complete records from load_am_auto_travel_time
8190 complete records from load_pm_auto_travel_time
8190 complete records from load_off_peak_auto_travel_time
8190 complete records from load_m25_stations
8190 complete records from load_median_hh_income
8190 complete records from load_nine_to_five_jobs
8190 complete records from load_night_and_weekend_jobs
8190 complete records from load_school_jobs
8190 complete records from load_sop
8190 complete records from calculate_interactions


In [88]:
df.to_csv('chester_modeling_df.csv')

In [89]:
flows = 'total_ridership_od'
o_vars = [
    'trains_per_hr_peak_o',
    'proportion_houses_o',
    'total_households_within_half_mi_o',
    'parking_capacity_o',
    'bus_line_count_o',
    'bus_stop_count_o',
    'total_jobs_within_half_mi_o',
    'miles_to_core_o',
    'terminal_dummy_2023_o',
    'm25_station_o',
    'median_hh_income_o',
    'nine_to_five_jobs_o',
    'night_weekend_jobs_o',
    'school_jobs_o',
    'sop_form_norm_o',
    'sop_dens_norm_o',
    'sop_prox_norm_o',
    'sop_conn_norm_o',
    'sop_park_norm_o',
    'sop_peds_norm_o',
    'sop_safe_norm_o',
    'sop_traf_norm_o',
    'sop_aest_norm_o',
    'sop_recr_norm_o',
    'sop_totl_norm_o',
]
d_vars = [
    'trains_per_hr_peak_d',
    'proportion_houses_d',
    'total_households_within_half_mi_d',
    'parking_capacity_d',
    'bus_line_count_d',
    'bus_stop_count_d',
    'total_jobs_within_half_mi_d',
    'miles_to_core_d',
    'terminal_dummy_2023_d',
    'm25_station_d',
    'median_hh_income_d',
    'nine_to_five_jobs_d',
    'night_weekend_jobs_d',
    'school_jobs_d',
    'sop_form_norm_d',
    'sop_dens_norm_d',
    'sop_prox_norm_d',
    'sop_conn_norm_d',
    'sop_park_norm_d',
    'sop_peds_norm_d',
    'sop_safe_norm_d',
    'sop_traf_norm_d',
    'sop_aest_norm_d',
    'sop_recr_norm_d',
    'sop_totl_norm_d',
]
od_vars = [
    'composite_miles_od',
    # 'peak_fare_od',
    # 'off_peak_fare_od',
    'sd_fare_od',
    'travel_time_od',
    'track_miles_od',
    # 'am_parking_user_od',
    # 'pm_parking_user_od',
    # 'off_peak_parking_user_od',
    'total_parking_user_od',
    'bus_tt_od',
    'bus_transfers_od',
    'am_auto_tt_od',
    # 'pm_auto_tt_od',
    # 'off_peak_auto_tt_od',
    'm25_station_o*m25_station_d',
]

# flows, o_vars, d_vars, od_vars = inputs(df, flows, o_vars, d_vars, od_vars)

estimate_models(df, flows, o_vars, d_vars, od_vars)

Gravity             OLS       
                                         coef    P       coef      P
const                                  -92.55  0.0 -103622.99   0.03
trains_per_hr_peak_o                     0.11  0.0       0.79  0.013
proportion_houses_o                     -0.34  0.0     -30.76  0.002
total_households_within_half_mi_o        0.72  0.0       0.04    0.0
parking_capacity_o                       -0.0  0.0      -0.01  0.589
...                                       ...  ...        ...    ...
am_auto_tt_od                            0.01  0.0       0.47  0.512
m25_station_o*m25_station_d             -0.45  0.0     328.28    0.0
AIC                                1208213.86       122187.96       
pseudoR2                                 0.74                       
R2                                                       0.45       

[63 rows x 4 columns]